# Domain Modelling Made Functional

## Understanding Types

## Functions

* Functions are the basis of _functional_ programming
* They take inputs, and produce outputs
* For any input, the output is *always* the same (we call this a pure function, but this is a what a function really is by its formal definition)
* An example might be some function that turns an apple into a banana (weird flex, but okay)

    Function apple -> banana

#### Type Signatures

* Type signatures just talk about what value the function represents

In [38]:
let addOne x = x + 1;;
// val addOne : x:int -> int

* You'll notice there's no return -> things are just expressions (value-returning), and the last expression is what is returned. Much like `clojure`

In [39]:
// Let's see what happens here!
let addOneWithMore x = 
  1 + 1
  x + 1

addOneWithMore 10;;

11

* What's fun and worth noting is that we are yet to have to tell our functions what the *types* of the inputs are. That's down to very clever *type inference*
* We will likely dive more deeply into that soon.
* We've somewhat seen what functions on many lines look like, but here's another example

In [40]:
let squareAndInc x = 
  let square x = x * x
  square x + 1

squareAndInc 10

101

* Revisiting a little bit of inference, sometimes functions will work on any type (likely with a condition) -> we can use generics here

In [41]:
let areEqual x y =
    x = y
// val areEqual : x:'a -> y:'a -> bool when 'a : equality

areEqual 10 11
areEqual 10 10.

Error: input.fsx (6,13)-(6,16) typecheck error This expression was expected to have type
    'int'    
but here has type
    'float'    

* Hey so this is kinda cool, we're getting a little bit more information here in our type signature!
* We are explicitly told that this function only works on some `'a`, implying both arguments must be of type `'a`
* Just in case it wasn't obvious, `'a` is the same as TS/Java/C# using `<T>`

## Types & Functions

* Types **are not** classes, they are simply names given to sets of *possible values*
* Consider that the type `int16` is just the name given to numbers that can be represented by 16 bits!
    * `- ( 2 ^ 15 ) -> (( 2 ^ 15 ) - 1))`
    * -32,768 -> 32,767
* What's cool is that's where it ends. There's no magic. That's the type.
* Types don't have to be primitives. They can be called anything!

In [42]:
type Fruit =
    | Apple
    | Pear
    | Banana

### Values vs Objects vs Variables

* It's the us and them babi
* In fp, most things are *values*
* A value, is just a *member* of a type (think about `int16`) from before
* Functions can be values too! Think about our fruit function from above, it's just a value whose type is `apple -> banana`
* Values are immutable (hence not called *variables*)
* Objects are just encapsulation of some data structure and its associated behaviour (methods)
* Generally speaking, objects are expected to have state, and operations that change this state must be provided by methods (dot access) -> ie Java class

## Composition of Types

* You'll hear the word composition a lot in functional programming -> it simply means combining *things* to get new *things* -> just like lego blocks
* In F#, new types are built from smaller types in two key ways
    * `and`
    * `or`

### `And`
* Typically looks like a record. We have things that go together

### `Or`
* The more complex option
* Consider these "choices", like "one of"

```fsharp
type FruitSnack = 
  | Apple of AppleVariety
  | Banana of BananaVariety
  | Cherries of CherryVariety
```

* This option, or choice is called a *discriminated union*

### Product and Sum Types
* If types are combined together using "and", then these are a `product` type
* If they are "ored" then these are `sum` types or `tagged unions`
  * Scotty The Flash Wlaschin will call them `choice` types, like a sensible person
  * I personally Find these names (not Scotty's) kinda counter-intuitve as sum in mathematics implies adding, and product multiplication, but anyway

### Working with types in F#

#### Algebraic Types
* Literally just a type system where each type is composed from smaller types

#### Worked Examples

In [43]:
type ProductCode = ProductCode of string // Gives us a "wrapper" to tell us its value!

// As an extension, let's get the raw value back
let productValue p = 
  let (ProductCode v) = p
  $"The value of the product is {v}";;

In [44]:
// This is a bit fun, but we can use the type 'constructor'
"Wazooki" |> ProductCode |> productValue;;

The value of the product is Wazooki

In [45]:
type Person = {first: String; last: String}

// look at more of that juicy inference!
let printPerson p = 
  $"Hello, {p.first} {p.last}!"

printPerson {first = "Rich"; last = "Hickey"};;

Hello, Rich Hickey!

In [46]:
type OrderQuantity = 
  | UnitQuantity of int
  | KilogramQuantity of decimal

let printQuantity q = 
    match q with
    | UnitQuantity u ->
        if 10 = u then "A perfect 10 units" else $"You ordered {u} units"
    | KilogramQuantity k ->
        if 10.0 |> decimal = k then "Might as well have just ordered a decimal" else $"You ordered {k}kg"

printQuantity (27.0 |> decimal |> KilogramQuantity);;

You ordered 27kg

#### A real Breakdown of the value of types

In [47]:
type CheckNumber = int;;
type CardNumber = string;;

type CreditCardType = Visa | MasterCard;;

type CreditCard = {CardType: CreditCardType; Number: CardNumber };;

type PaymentMethod = 
  | Cash
  | Check of CheckNumber
  | Card of CreditCard

type PaymentAmount = decimal;;

type Currency = EUR | GBP | AUD

type Payment = {Amount: PaymentAmount; Currency: Currency; Method: PaymentMethod}

#### Extra Bits - Options, missing values & more

```fsharp
type ExampleOption = {A: Option<string>} // exact same as {A: string option}, with the latter being more idiomatic
type ExampleList = {A: List<string>} // and again {A: string list}, again, the latter is more idiomatic
```

##### None

```fsharp
type ExampleNone = {A: None}
```

But what about the absence of anyting? Much like `void`?

```fsharp
type NothingAtAll = unit
```

##### Errors
* As of F# 4.1, there's a built in `Result` type, but let's derive it using things we've done before!



In [48]:
type FunctionResult<'Ok, 'Error> = 
  | Success of 'Ok
  | Failure of 'Error

##### Lists

* Lists are much the same as in clojure
* We can destructure on them with pattern matches
* `cons` is a very common operation which adds to the head of a list and is a "constant time" operation

### Code & Type Organisation
* Just do it in order. 
    * You technically don't have to, and you can use the `rec` keyword, but just don't be that guy imo

## Random Extenion We Can Hack Together

In [49]:
let toString fruit =
    match fruit with
    | Apple -> "I am an apple"
    | Pear -> "I am a pear"
    | Banana -> "I am inferior and sad"


let toFruit s =
    match s with
    | "Apple" -> Apple
    | "Pear" -> Pear
    | "Banana" -> Banana
    | _ -> failwith "Poo!";;
    

In [50]:
["Apple" ; "Pear" ; "Banana"] |> List.map (toFruit >> toString) |> string;;

[I am an apple; I am a pear; I am inferior and sad]

> Here's something we might be more familiar with!

```clojure
; Note that in f# we get to write our functions "in order", but in clojure we work "outwards"
(->> ["Apple "Pear" "Banana"]
     (map (comp toString toFruit))
     (str))
```

In [51]:
// What about a potato?!
(toFruit >> toString) "Potato";;

Error: System.Exception: Poo!
   at FSI_0062.toFruit(String s)
   at <StartupCode$FSI_0064>.$FSI_0064.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodBaseInvoker.InvokeWithNoArgs(Object obj, BindingFlags invokeAttr)

In [52]:
let prependFruit fruit fruits: Fruit list = fruit :: fruits

["Apple" ; "Pear"] |> List.map toFruit |> prependFruit ("Banana" |> toFruit) |> string

[Banana; Apple; Pear]